In [ ]:
import geopandas
gdf = geopandas.read_file('data/pocndvi.shp')
gdf.tail()

In [ ]:
mask = gdf['gisId'] == '32109'
polygon = gdf[mask]['geometry'].__geo_interface__["features"][0]["geometry"]


In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("/Users/tammerkamel/dev/proj/Streambatch/src/Streambatch/"))

from module1 import StreambatchConnection

In [ ]:
connection = StreambatchConnection(open('key.txt').read().strip())
query_id = connection.request_ndvi(
            polygons=[polygon],
            sources=['ndvi.savgol'],
            aggregation='mean',
            query_id='9e2908aa-a5c0-4822-b3d0-96bbe81d5c7b'
            )

In [ ]:
df = connection.get_data(query_id)

In [ ]:
import matplotlib.pyplot as plt

# slice df for the last two years
df1 = df[(df['time'] > '2022-01-01') & (df['time'] < '2023-12-27')]

# scatter plot time vs ndvi.savgol with s=3 point size
# plt.figure(figsize=(30, 15))

plt.figure(figsize=(30, 15))
plt.rcParams.update({'font.size': 22})

plt.scatter(df1['time'], df1['ndvi.savgol'], label='savgol', s=13)
# plt.scatter(df1['time'], df1['ndvi.interpolated'], label='ndvi',s=13)
plt.scatter(df1['time'], df1['ndvi.landsat'], label='landsat',s=63,color='green')
plt.scatter(df1['time'], df1['ndvi.sentinel2'], label='sentinel-2',s=63,color='red')
# labels

plt.grid(True)
plt.ylim(0, 1.)

plt.xlabel('time')
plt.ylabel('ndvi')
plt.legend()



plt.show()